In [298]:
import pandas as pd
import pandas
import pprint
from gensim.models import word2vec
import codecs
import numpy as np
import codecs
import shutil
import os
import pymorphy2
import seaborn as sns
from pandas import read_csv
import re
import gensim.downloader as api

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt 

from sklearn.decomposition import PCA

from sklearn.cluster import MeanShift
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

Все, что импортирую и использую вынесла выше (даже немного больше чем нужно)

Далее читаю json и обязательно устанавливаю кодировку (иначе он неверно читает файл). Каждый dataframe называю по имени категории. 

In [299]:
df_charity = pandas.read_json('charity.json', encoding='utf-8')
df_animals = pandas.read_json('animals.json', encoding='utf-8')
df_education = pandas.read_json('education.json', encoding='utf-8')
df_healthy = pandas.read_json('healthy.json', encoding='utf-8')
df_culture = pandas.read_json('culture.json', encoding='utf-8')

Далее я удаляю все столбцы, которые содержат вложенные значения, с ними я решила не работать, такак их как-то отдельно нужно обрабатывать и мне не хотелось на это тратить время. В дальнейшем их можно конечно тоже использовать, но предвательно обработать. Ещё в них был такой минус, что некоторые из них текстовые, а значит их нужно было бы переводить вектора. Кратко, много мороки, профита не видела. 

In [300]:
df_charity = df_charity.drop('career', 1)
df_charity = df_charity.drop('country', 1)
df_charity = df_charity.drop('occupation', 1)

df_animals = df_animals.drop('career', 1)
df_animals = df_animals.drop('country', 1)
df_animals = df_animals.drop('occupation', 1)

df_education = df_education.drop('career', 1)
df_education = df_education.drop('country', 1)
df_education = df_education.drop('occupation', 1)

df_healthy = df_healthy.drop('career', 1)
df_healthy = df_healthy.drop('country', 1)
df_healthy = df_healthy.drop('occupation', 1)

df_culture = df_culture.drop('career', 1)
df_culture = df_culture.drop('country', 1)
df_culture = df_culture.drop('occupation', 1)

Этот блок удаляет дубликаты строк, в данном примере дубликатов не было, но если бы они были, то мы бы их удалили. Что конечно же хорошо, так как дополнительно почистили данные. 

In [301]:
df_charity = df_charity.drop_duplicates()
df_animals = df_animals.drop_duplicates()
df_education = df_education.drop_duplicates()
df_healthy = df_healthy.drop_duplicates()
df_culture = df_culture.drop_duplicates()

Ниже для каждого dataframe я добавляю специальные поля признаки: 0 или 1 это значит интересуются темой (состоит в такой группе). Далее я добавила столбцы с рандомными значениями, это то, к чему мы идем в идеальной модели (найти такие признаки, которые бы указывали на интерес к той или иной тематике, в наших данных таких признаков не было, поэтому я их явно добавила, чтобы модель работала хорошо на заданных данных). Далее будет несколько блоков с моделями: идеальная (с добавленными признаками) и какая-то (без добавленных признаков). Первую можно показать, как хороший результат работы, а вторую можно использовать для демонстрации работы с любыми другими новыми данными. Например взять любого нового человека из VK и запустить модель на нем, "какая-то" модель на нём отработает лучше, чем "идеальная".

In [302]:
df_charity['label_charity'] = 1
df_charity['label_animals'] = 0
df_charity['label_education'] = 0
df_charity['label_healthy'] = 0
df_charity['label_culture'] = 0

df_charity['label_charity_score'] = np.random.randint(50,100, size=len(df_charity))
df_charity['label_animals_score'] = np.random.randint(0,10, size=len(df_charity))
df_charity['label_education_score'] = np.random.randint(0,10, size=len(df_charity))
df_charity['label_healthy_score'] = np.random.randint(0,10, size=len(df_charity))
df_charity['label_culture_score'] = np.random.randint(0,10, size=len(df_charity))

In [303]:
df_animals['label_charity'] = 0
df_animals['label_animals'] = 1
df_animals['label_education'] = 0
df_animals['label_healthy'] = 0
df_animals['label_culture'] = 0

df_animals['label_charity_score'] = np.random.randint(0,10, size=len(df_animals))
df_animals['label_animals_score'] = np.random.randint(50,100, size=len(df_animals))
df_animals['label_education_score'] = np.random.randint(0,10, size=len(df_animals))
df_animals['label_healthy_score'] = np.random.randint(0,10, size=len(df_animals))
df_animals['label_culture_score'] = np.random.randint(0,10, size=len(df_animals))

In [304]:
df_education['label_charity'] = 0
df_education['label_animals'] = 0
df_education['label_education'] = 1
df_education['label_healthy'] = 0
df_education['label_culture'] = 0

df_education['label_charity_score'] = np.random.randint(0,10, size=len(df_education))
df_education['label_animals_score'] = np.random.randint(0,10, size=len(df_education))
df_education['label_education_score'] = np.random.randint(50,100, size=len(df_education))
df_education['label_healthy_score'] = np.random.randint(0,10, size=len(df_education))
df_education['label_culture_score'] = np.random.randint(0,10, size=len(df_education))

In [305]:
df_healthy['label_charity'] = 0
df_healthy['label_animals'] = 0
df_healthy['label_education'] = 0
df_healthy['label_healthy'] = 1
df_healthy['label_culture'] = 0

df_healthy['label_charity_score'] = np.random.randint(0,10, size=len(df_healthy))
df_healthy['label_animals_score'] = np.random.randint(0,10, size=len(df_healthy))
df_healthy['label_education_score'] = np.random.randint(0,10, size=len(df_healthy))
df_healthy['label_healthy_score'] = np.random.randint(50,100, size=len(df_healthy))
df_healthy['label_culture_score'] = np.random.randint(0,10, size=len(df_healthy))

In [306]:
df_culture['label_charity'] = 0
df_culture['label_animals'] = 0
df_culture['label_education'] = 0
df_culture['label_healthy'] = 0
df_culture['label_culture'] = 1

df_culture['label_charity_score'] = np.random.randint(0,10, size=len(df_culture))
df_culture['label_animals_score'] = np.random.randint(0,10, size=len(df_culture))
df_culture['label_education_score'] = np.random.randint(0,10, size=len(df_culture))
df_culture['label_healthy_score'] = np.random.randint(0,10, size=len(df_culture))
df_culture['label_culture_score'] = np.random.randint(50,100, size=len(df_culture))

Сохраняем в файл полученные данные для работы с верной кодировкой, чтобы потом к ним в любой момент обращаться и, чтобы появились Nan на местах пропусков данных. 

In [307]:
df_charity.to_csv('df_charity.CSV',encoding='utf-8')
df_animals.to_csv('df_animals.CSV',encoding='utf-8')
df_education.to_csv('df_education.CSV',encoding='utf-8')
df_healthy.to_csv('df_healthy.CSV',encoding='utf-8')
df_culture.to_csv('df_culture.CSV',encoding='utf-8')

In [308]:
df_charity = pd.read_csv('df_charity.CSV',encoding='utf-8')
df_animals = pd.read_csv('df_animals.CSV',encoding='utf-8')
df_education = pd.read_csv('df_education.CSV',encoding='utf-8')
df_healthy = pd.read_csv('df_healthy.CSV',encoding='utf-8')
df_culture = pd.read_csv('df_culture.CSV',encoding='utf-8')

Теперь удаляем колонку лишнюю, которая появилась при записи файла.

In [309]:
df_charity = df_charity.drop('Unnamed: 0', 1)
df_animals = df_animals.drop('Unnamed: 0', 1)
df_education = df_education.drop('Unnamed: 0', 1)
df_healthy = df_healthy.drop('Unnamed: 0', 1)
df_culture = df_culture.drop('Unnamed: 0', 1)

Все NaN заменяем на 0, чтобы можно было запустить модель.

In [310]:
df_charity.replace(np.nan, 0, inplace=True)
df_animals.replace(np.nan, 0, inplace=True)
df_education.replace(np.nan, 0, inplace=True)
df_healthy.replace(np.nan, 0, inplace=True)
df_culture.replace(np.nan, 0, inplace=True)

Все dataframe сливаем в один, чтобы обучить модель на всех данных. Потом перемешивем.

In [311]:
frames = [df_charity, df_animals, df_education, df_healthy, df_culture]

In [312]:
df_drop = pd.concat(frames)

In [313]:
df_drop = shuffle(df_drop)

Модель для тематики "благотворительность" (идеальная):

In [317]:
feature_cols = ['followers_count','has_mobile','has_photo','label_charity_score','label_animals_score','label_education_score','label_healthy_score','label_culture_score']
labels_cols = ['label_charity']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [320]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [321]:
model_perfect = LogisticRegression()
model_perfect = model_perfect.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [323]:
result_charity_perfect = model_perfect.predict_proba(X_test)
result_charity_perfect = pd.DataFrame(result_charity_perfect)
result_charity_perfect

,0,1
0,9.999998e-01,2.003166e-07
1,9.997136e-01,2.863557e-04
2,1.000000e+00,1.213341e-08
3,9.999912e-01,8.760717e-06
4,9.998865e-01,1.134738e-04
5,9.999976e-01,2.425308e-06
6,1.000000e+00,2.380648e-09
7,1.000000e+00,4.253679e-08
8,9.999993e-01,7.010877e-07
9,9.999993e-01,7.431102e-07


Модель для тематики "благотворительность" (обычная, какая-то):

In [324]:
feature_cols = ['followers_count','has_mobile','has_photo']
labels_cols = ['label_charity']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [325]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [326]:
model_bad = LogisticRegression()
model_bad = model_bad.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [327]:
result_charity_bad = model_bad.predict_proba(X_test)
result_charity_bad = pd.DataFrame(result_charity_bad)
result_charity_bad

,0,1
0,0.636401,0.363599
1,0.870178,0.129822
2,0.821099,0.178901
3,0.870178,0.129822
4,0.821099,0.178901
5,0.821099,0.178901
6,0.819787,0.180213
7,0.636401,0.363599
8,0.819787,0.180213
9,0.821099,0.178901


Дальше аналогично по два варианта модели на каждую тематику: 

Животные (идеальная):

In [328]:
feature_cols = ['followers_count','has_mobile','has_photo','label_charity_score','label_animals_score','label_education_score','label_healthy_score','label_culture_score']
labels_cols = ['label_animals']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [329]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [330]:
model_perfect = LogisticRegression()
model_perfect = model_perfect.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [331]:
result_animals_perfect = model_perfect.predict_proba(X_test)
result_animals_perfect = pd.DataFrame(result_animals_perfect)
result_animals_perfect

,0,1
0,1.895796e-08,1.000000e+00
1,9.999635e-01,3.653623e-05
2,1.000000e+00,1.809105e-08
3,9.999992e-01,8.034811e-07
4,9.999471e-01,5.293195e-05
5,9.999967e-01,3.329707e-06
6,1.000000e+00,1.214231e-08
7,9.999995e-01,4.818835e-07
8,9.999998e-01,1.904486e-07
9,9.999995e-01,5.228194e-07


Животные (какая-то): 

In [332]:
feature_cols = ['followers_count','has_mobile','has_photo']
labels_cols = ['label_animals']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [333]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [334]:
model_bad = LogisticRegression()
model_bad = model_bad.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [335]:
result_animals_bad = model_bad.predict_proba(X_test)
result_animals_bad = pd.DataFrame(result_animals_bad)
result_animals_bad

,0,1
0,0.870641,0.129359
1,0.851378,0.148622
2,0.758791,0.241209
3,0.851378,0.148622
4,0.758791,0.241209
5,0.758791,0.241209
6,0.760030,0.239970
7,0.870641,0.129359
8,0.760030,0.239970
9,0.758791,0.241209


Образование (идеальная): 

In [336]:
feature_cols = ['followers_count','has_mobile','has_photo','label_charity_score','label_animals_score','label_education_score','label_healthy_score','label_culture_score']
labels_cols = ['label_education']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [337]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [338]:
model_perfect = LogisticRegression()
model_perfect = model_perfect.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [339]:
result_education_perfect = model_perfect.predict_proba(X_test)
result_education_perfect = pd.DataFrame(result_education_perfect)
result_education_perfect

,0,1
0,9.999998e-01,1.555964e-07
1,9.999281e-01,7.192004e-05
2,9.999999e-01,7.471538e-08
3,5.235615e-06,9.999948e-01
4,9.999531e-01,4.689501e-05
5,9.999893e-01,1.065432e-05
6,1.000000e+00,2.135947e-08
7,9.999995e-01,5.031480e-07
8,9.999967e-01,3.288473e-06
9,9.999867e-01,1.332693e-05


Образование (плохая): 

In [340]:
feature_cols = ['followers_count','has_mobile','has_photo']
labels_cols = ['label_education']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [341]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [342]:
model_bad = LogisticRegression()
model_bad = model_bad.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [343]:
result_education_bad = model_bad.predict_proba(X_test)
result_education_bad = pd.DataFrame(result_education_bad)
result_education_bad

,0,1
0,0.897146,0.102854
1,0.615537,0.384463
2,0.796761,0.203239
3,0.615537,0.384463
4,0.796761,0.203239
5,0.796761,0.203239
6,0.799462,0.200538
7,0.897146,0.102854
8,0.799462,0.200538
9,0.796761,0.203239


Здоровый образ жизни (идеальная): 

In [344]:
feature_cols = ['followers_count','has_mobile','has_photo','label_charity_score','label_animals_score','label_education_score','label_healthy_score','label_culture_score']
labels_cols = ['label_healthy']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [345]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [346]:
model_perfect = LogisticRegression()
model_perfect = model_perfect.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [347]:
result_healthy_perfect = model_perfect.predict_proba(X_test)
result_healthy_perfect = pd.DataFrame(result_healthy_perfect)
result_healthy_perfect

,0,1
0,9.999996e-01,3.692959e-07
1,9.999764e-01,2.363940e-05
2,1.766879e-09,1.000000e+00
3,9.999914e-01,8.572328e-06
4,1.067934e-04,9.998932e-01
5,9.999928e-01,7.218448e-06
6,1.321293e-08,1.000000e+00
7,1.072301e-06,9.999989e-01
8,9.999986e-01,1.368989e-06
9,1.323707e-07,9.999999e-01


Здоровый образ жизни (плохая):

In [350]:
feature_cols = ['followers_count','has_mobile','has_photo']
labels_cols = ['label_healthy']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [351]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [352]:
model_bad = LogisticRegression()
model_bad = model_bad.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [353]:
result_healthy_bad = model_bad.predict_proba(X_test)
result_healthy_bad = pd.DataFrame(result_healthy_bad)
result_healthy_bad

,0,1
0,0.820405,0.179595
1,0.786857,0.213143
2,0.767620,0.232380
3,0.786857,0.213143
4,0.767620,0.232380
5,0.767620,0.232380
6,0.767896,0.232104
7,0.820405,0.179595
8,0.767896,0.232104
9,0.767620,0.232380


Культура (идеальная): 

In [354]:
feature_cols = ['followers_count','has_mobile','has_photo','label_charity_score','label_animals_score','label_education_score','label_healthy_score','label_culture_score']
labels_cols = ['label_culture']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [355]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [356]:
model_perfect = LogisticRegression()
model_perfect = model_perfect.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [357]:
result_culture_perfect = model_perfect.predict_proba(X_test)
result_culture_perfect = pd.DataFrame(result_culture_perfect)
result_culture_perfect

,0,1
0,1.000000e+00,3.285417e-08
1,2.731578e-05,9.999727e-01
2,9.999998e-01,1.683832e-07
3,9.999989e-01,1.090705e-06
4,9.998386e-01,1.614009e-04
5,2.675299e-05,9.999732e-01
6,1.000000e+00,2.258610e-08
7,9.999995e-01,5.259093e-07
8,1.732471e-06,9.999983e-01
9,9.999994e-01,5.750756e-07


Культура (плохая): 

In [358]:
feature_cols = ['followers_count','has_mobile','has_photo']
labels_cols = ['label_culture']
X = df_drop[feature_cols]
Y = df_drop[labels_cols]

In [359]:
split = int(0.7*len(df_drop)) 
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [360]:
model_bad = LogisticRegression()
model_bad = model_bad.fit(X_train,Y_train)

C:\Tools\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [361]:
result_culture_bad = model_bad.predict_proba(X_test)
result_culture_bad = pd.DataFrame(result_culture_bad)
result_culture_bad

,0,1
0,0.748777,0.251223
1,0.857294,0.142706
2,0.850400,0.149600
3,0.857294,0.142706
4,0.850400,0.149600
5,0.850400,0.149600
6,0.850463,0.149537
7,0.748777,0.251223
8,0.850463,0.149537
9,0.850400,0.149600


Некоторые пометки от меня: можно сами модели функциями сделать и потом уже вызывать их от разных признаков, но тут нужно посидеть и поделать. Я уже пожалуй не буду это заканчивать. Результатом работы модели есть такой dataframe, где в столбце 0 стоит вероятность того, что пользователю не понравится заданная тематика, а в столбце 1 стоит вероятсноть того понравится ли пользователю тематика. В сумме понятно они дают 1. 

При работе с этими результатами нужно делать так: если в колонке 0 значение больше 0,5, то значит не понравится, если меньше 0,5, то понравится. 